In [36]:
import os
from dotenv import load_dotenv

In [37]:
load_dotenv()

True

In [38]:
import openai

In [39]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [40]:
import pandas as pd

In [41]:
df = pd.read_csv("sales_data_sample.csv")

In [42]:
df

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,10350,20,100.00,15,2244.40,12/2/2004 0:00,Shipped,4,12,2004,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Small
2819,10373,29,100.00,1,3978.51,1/31/2005 0:00,Shipped,1,1,2005,...,Torikatu 38,NaN,Oulu,NaN,90110,Finland,EMEA,Koskitalo,Pirkko,Medium
2820,10386,43,100.00,4,5417.57,3/1/2005 0:00,Resolved,1,3,2005,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Medium
2821,10397,34,62.24,1,2116.16,3/28/2005 0:00,Shipped,1,3,2005,...,1 rue Alsace-Lorraine,NaN,Toulouse,NaN,31000,France,EMEA,Roulet,Annette,Small


In [12]:
# df.groupby("QTR_ID").sum()['SALES']

In [43]:
import sqlalchemy

In [44]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [45]:
# Create a temp DB in Ram
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [46]:
# Push Pandas DF --> Temp DB
data = df.to_sql(name='Sales', con=temp_db)

2023-05-24 16:06:13,943 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-24 16:06:13,952 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-05-24 16:06:13,953 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-24 16:06:13,957 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-05-24 16:06:13,958 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-24 16:06:13,962 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

In [47]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT SUM(SALES) FROM Sales"))

2023-05-24 16:06:22,970 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-24 16:06:22,971 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales
2023-05-24 16:06:22,972 INFO sqlalchemy.engine.Engine [generated in 0.00207s] ()
2023-05-24 16:06:22,973 INFO sqlalchemy.engine.Engine ROLLBACK


In [48]:
result.all()

[(10032628.85000001,)]

In [49]:
def create_table_definition(df):
    promt = """### sqlite SQL table, with it properties:
    #
    # Sales({})
    #
    """.format(",".join(str(col) for col in df.columns))

    return promt

In [50]:
",".join(str(col) for col in df.columns)

'ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE'

In [51]:
print(create_table_definition(df))

### sqlite SQL table, with it properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    


In [52]:
def prompt_input():
    nlp_text=input("Enter the info you want: ")
    return nlp_text

In [53]:
prompt_input()

Enter the info you want: 


''

In [54]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition + query_init_string

In [61]:
# nlp_text = prompt_input()
nlp_text = "return the sum of SALES per POSTALCODE"
combine_prompts(df, nlp_text)

'### sqlite SQL table, with it properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    #\n    ### A query to answer: return the sum of SALES per POSTALCODE\nSELECT'

In [58]:
response = openai.Completion.create(
    model = 'text-davinci-003',
    prompt = combine_prompts(df, nlp_text),
    temperature = 0,
    max_tokens = 150,
    top_p =1.0,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = ['#', ';']
)

In [62]:
response['choices'][0]['text']

' POSTALCODE, SUM(SALES)\nFROM Sales\nGROUP BY POSTALCODE'

In [63]:
def handle_response(response):
    query = response['choices'][0]['text']
     if query.startswith(" "):
        query = "SELECT"+query
    return query

In [64]:
handle_response(response)

'SELECT POSTALCODE, SUM(SALES)\nFROM Sales\nGROUP BY POSTALCODE'

In [65]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-05-24 16:16:01,274 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-24 16:16:01,276 INFO sqlalchemy.engine.Engine SELECT POSTALCODE, SUM(SALES)
FROM Sales
GROUP BY POSTALCODE
2023-05-24 16:16:01,276 INFO sqlalchemy.engine.Engine [generated in 0.00197s] ()
2023-05-24 16:16:01,279 INFO sqlalchemy.engine.Engine ROLLBACK


In [66]:
result.all()

[(None, 272407.14),
 ('10022', 560787.7699999998),
 ('10100', 94117.26000000002),
 ('106-0032', 120562.73999999996),
 ('1203', 117713.55999999998),
 ('1227 MM', 94015.73),
 ('13008', 74936.14),
 ('1734', 145041.6),
 ('2', 57756.43),
 ('2060', 153996.13000000003),
 ('2067', 151570.98000000004),
 ('21240', 111250.37999999996),
 ('24067', 85555.98999999998),
 ('24100', 137955.72000000003),
 ('28023', 170257.33000000005),
 ('28034', 912294.1100000002),
 ('3004', 200995.40999999997),
 ('31000', 70488.44),
 ('3150', 64591.46000000001),
 ('4101', 59469.11999999999),
 ('4110', 116599.19),
 ('41101', 54723.62),
 ('42100', 142601.33000000002),
 ('44000', 204304.86),
 ('5020', 149798.63),
 ('50553', 207874.86),
 ('50739', 100306.58),
 ('51003', 154069.65999999997),
 ('51100', 135042.94),
 ('51247', 139243.99999999994),
 ('530-0003', 67605.07),
 ('58339', 165255.20000000004),
 ('59000', 69052.41),
 ('60528', 85171.58999999998),
 ('62005', 131685.30000000002),
 ('67000', 80438.48),
 ('69004', 14287